<h1>Health condition detector NB</h1>

this part was implemented and done by Abdalla aka @qxisa on github. find more info through the readme on the repo link

credits go to their rightful owners, the following dataset and all the datasets that are implemented and used in this notebook are from Kaggle by Choong Qian Zheng & Pranay Patil under a CC license.



In [3]:
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score


In [4]:
df = pd.read_csv("dataset.csv").fillna("none")

df

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,none,none,none,none,none,none,none,none,none,none,none,none,none
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,none,none,none,none,none,none,none,none,none,none,none,none,none,none
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,none,none,none,none,none,none,none,none,none,none,none,none,none,none
3,Fungal infection,itching,skin_rash,dischromic _patches,none,none,none,none,none,none,none,none,none,none,none,none,none,none
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,none,none,none,none,none,none,none,none,none,none,none,none,none,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,(vertigo) Paroymsal Positional Vertigo,vomiting,headache,nausea,spinning_movements,loss_of_balance,unsteadiness,none,none,none,none,none,none,none,none,none,none,none
4916,Acne,skin_rash,pus_filled_pimples,blackheads,scurring,none,none,none,none,none,none,none,none,none,none,none,none,none
4917,Urinary tract infection,burning_micturition,bladder_discomfort,foul_smell_of urine,continuous_feel_of_urine,none,none,none,none,none,none,none,none,none,none,none,none,none
4918,Psoriasis,skin_rash,joint_pain,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,none,none,none,none,none,none,none,none,none,none,none


In [5]:
symptom_cols = df.columns[1:] #all except disease

all_symptoms = pd.unique(df[symptom_cols].values.ravel())
all_symptoms = [symptom for symptom in all_symptoms if symptom != "none"]




In [6]:
X = pd.DataFrame(0, index=df.index, columns=all_symptoms)

for i, row in df.iterrows():
    for symptom_col in symptom_cols:
        symptom = row[symptom_col]
        if symptom != "none":
            X.at[i, symptom] = 1



In [7]:
le = LabelEncoder()
y = le.fit_transform(df["Disease"])


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=50)
model = MultinomialNB()
model.fit(X_train, y_train)


y_pred = model.predict(X_test)
print(f"Model accuracy: {accuracy_score(y_test, y_pred):.2f}")


Model accuracy: 1.00


In [9]:
precautions_df = pd.read_csv("symptom_precaution.csv")
descriptions_df = pd.read_csv("symptom_Description.csv")
severity_df = pd.read_csv("Symptom-severity.csv")


precautions_df.columns = [col.strip().capitalize() for col in precautions_df.columns]
descriptions_df.columns = [col.strip().capitalize() for col in descriptions_df.columns]
severity_df.columns = [col.strip().capitalize() for col in severity_df.columns]


In [10]:
def predict_disease(symptoms):
    # Prepare input vector
    input_vec = pd.DataFrame(0, index=[0], columns=X.columns)
    for s in symptoms:
        s_clean = s.strip().lower()
        # Match symptom ignoring case
        for col in X.columns:
            if col.lower() == s_clean:
                input_vec.at[0, col] = 1

    pred_encoded = model.predict(input_vec)[0]
    return le.inverse_transform([pred_encoded])[0]

def get_description(disease):
    row = descriptions_df[descriptions_df["Disease"].str.lower() == disease.lower()]
    if not row.empty:
        return row.iloc[0]["Description"]
    return "No description available."

def get_precautions(disease):
    row = precautions_df[precautions_df["Disease"].str.lower() == disease.lower()]
    if not row.empty:
        return [p for p in row.iloc[0, 1:].dropna() if p]
    return ["No precautions found."]

def get_severity(symptoms):
    result = []
    for s in symptoms:
        row = severity_df[severity_df["Symptom"].str.lower() == s.lower()]
        if not row.empty:
            result.append((s, row.iloc[0]["Weight"]))
        else:
            result.append((s, "Unknown"))
    return result


In [11]:


#user_symptoms = ['itching', 'skin_rash', 'nodal_skin_eruptions']
#user_symptoms = ['fever', 'dry_cough', 'fatigue', 'sore_throat', 'breathing_difficulty']
user_symptoms = ['fever', 'headache', 'nausea', 'vomiting', 'diarrhea']

predicted_disease = predict_disease(user_symptoms)
print(f"Predicted Disease: {predicted_disease}\n")

print("Description:")
print(get_description(predicted_disease), "\n")

print("Precautions:")
for p in get_precautions(predicted_disease):
    print("-", p)

print("\nSymptom Severity:")
for s, w in get_severity(user_symptoms):
    print(f"{s}: {w}")


Predicted Disease: hepatitis A

Description:
Hepatitis A is a highly contagious liver infection caused by the hepatitis A virus. The virus is one of several types of hepatitis viruses that cause inflammation and affect your liver's ability to function. 

Precautions:
- Consult nearest hospital
- wash hands through
- avoid fatty spicy food
- medication

Symptom Severity:
fever: Unknown
headache: 3
nausea: 5
vomiting: 5
diarrhea: Unknown


<h1>Neural network implementation</h1>

In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Convert labels to categorical for multi-class classification
y_cat = to_categorical(y)

# Split data (reuse same train/test split or recreate)
X_train, X_test, y_train_cat, y_test_cat = train_test_split(X, y_cat, test_size=0.2, random_state=42)

# Define model architecture
model_nn = Sequential([
    Dense(128, activation='relu', input_shape=(X.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(y_cat.shape[1], activation='softmax')
])

# Compile model
model_nn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
history = model_nn.fit(X_train, y_train_cat, epochs=30, batch_size=16, validation_split=0.1)

# Evaluate model
loss, accuracy = model_nn.evaluate(X_test, y_test_cat)
print(f"Neural Network accuracy: {accuracy:.2f}")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.4076 - loss: 2.8601 - val_accuracy: 1.0000 - val_loss: 0.1412
Epoch 2/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9876 - loss: 0.2097 - val_accuracy: 1.0000 - val_loss: 0.0110
Epoch 3/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9994 - loss: 0.0511 - val_accuracy: 1.0000 - val_loss: 0.0032
Epoch 4/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9983 - loss: 0.0304 - val_accuracy: 1.0000 - val_loss: 0.0014
Epoch 5/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9989 - loss: 0.0182 - val_accuracy: 1.0000 - val_loss: 7.5814e-04
Epoch 6/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9997 - loss: 0.0122 - val_accuracy: 1.0000 - val_loss: 3.5886e-04
Epoch 7/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9998 - loss: 0.0110 - val_accuracy: 1.0000 - val_loss: 2.1967e-04
Epoch 8/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9984 - loss: 0.0086 - val

In [13]:
import numpy as np

def predict_disease_nn(symptoms):
    input_vec = pd.DataFrame(0, index=[0], columns=X.columns)
    for s in symptoms:
        s_clean = s.strip().lower()
        for col in X.columns:
            if col.lower() == s_clean:
                input_vec.at[0, col] = 1

    preds = model_nn.predict(input_vec)
    pred_index = np.argmax(preds)
    return le.inverse_transform([pred_index])[0]

In [14]:
user_symptoms = ['itching', 'skin_rash', 'nodal_skin_eruptions']

predicted_disease = predict_disease_nn(user_symptoms)
print(f"Predicted Disease (NN): {predicted_disease}\n")

print("Description:")
print(get_description(predicted_disease), "\n")

print("Precautions:")
for p in get_precautions(predicted_disease):
    print("-", p)

print("\nSymptom Severity:")
for s, w in get_severity(user_symptoms):
    print(f"{s}: {w}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
Predicted Disease (NN): Fungal infection

Description:
In humans, fungal infections occur when an invading fungus takes over an area of the body and is too much for the immune system to handle. Fungi can live in the air, soil, water, and plants. There are also some fungi that live naturally in the human body. Like many microbes, there are helpful fungi and harmful fungi. 

Precautions:
- bath twice
- use detol or neem in bathing water
- keep infected area dry
- use clean cloths

Symptom Severity:
itching: 1
skin_rash: 3
nodal_skin_eruptions: 4
